In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from utils.data import load_tc_data
from utils.plot import make_cmap
import cftime as cf
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from netCDF4 import Dataset
from numpy import inf
# from utils.evaluation import find_landfalling_tcs,tc_region

In [ ]:
precip_cmap,precip_norm = make_cmap()
levels = [10, 15, 20, 25, 30, 40, 50,75, 100]

In [ ]:
# open storm dataset
tc_dir = '/user/home/al18709/work/event_sets/wgan_scalar/'
tc_dir = '/user/home/al18709/work/event_sets/wgan_modular/'
storm_filename = 'validation_mraw_2006237N13298.nc'
storm_filename2 = 'validation_1and2_2006237N13298.nc'
storm = xr.open_dataset(tc_dir + storm_filename)
storm_scores = np.load(tc_dir + 'validation_mraw_critic_2006237N13298.npy')

storm2 = xr.open_dataset(tc_dir + storm_filename2)
storm_scores2 = np.load(tc_dir + 'validation_1and2_critic_2006237N13298.npy')

tc_dir_truth = '/user/home/al18709/work/event_sets/truth/'
storm_truth = xr.open_dataset(tc_dir_truth + 'validation_2006237N13298.nc')

In [ ]:
# define initial variables
precip_cmap,precip_norm = make_cmap()

lats = storm.storm_lats[0,:,:]
lons = storm.storm_lons[0,:,:]

levels = [10, 15, 20, 25, 30, 40, 50,75, 100]
sns.set_style("white")
sns.set_context("notebook")
vmin=0
vmax=100

print(lats.shape)
print(lons.shape)


ntime,_,_,_ = storm.precipitation.shape
rain = storm.precipitation[0,:,:,0]
rain_truth = storm_truth.precipitation[0,:,:,0]


In [ ]:
# find storm extent
fp = '/bp1/geog-tropical/data/Obs/MSWEP/3hourly_invertlat/2000342.00.nc'
d = Dataset(fp, 'r')
lat = d.variables['lat'][:] #lat
lon = d.variables['lon'][:] #lon
lat_lower_bound = (np.abs(lat-np.min(storm.storm_lats))).argmin()
lat_upper_bound = (np.abs(lat-np.max(storm.storm_lats))).argmin()
lon_lower_bound = (np.abs(lon-np.min(storm.storm_lons))).argmin()
lon_upper_bound = (np.abs(lon-np.max(storm.storm_lons))).argmin()

lats = lat[lat_lower_bound:lat_upper_bound+2]
lons = lon[lon_lower_bound:lon_upper_bound+2]
grid_x, grid_y = np.meshgrid(lons,lats)
print(grid_x.shape)
print(grid_y.shape)

In [ ]:
# superimpose rain onto bigger grid
grid_rain = np.zeros((ntime,grid_x.shape[0],grid_y.shape[1],20))
grid_rain2 = np.zeros((ntime,grid_x.shape[0],grid_y.shape[1],20))
grid_rain_truth = np.zeros((ntime,grid_x.shape[0],grid_y.shape[1]))

for t in range(ntime):

	storm_lons = storm.storm_lons[t,:,:]
	storm_lats = storm.storm_lats[t,:,:]
	
	storm_rain_truth = storm_truth.precipitation[t,:,:,0]

	Mlon = storm_lons[-1,-1]
	mlon = storm_lons[0,0]
	Mlat = storm_lats[-1,-1]
	mlat = storm_lats[0,0]
	Xspan = np.where((grid_x <= Mlon) & (grid_x >= mlon))[1][[0, -1]]
	Yspan = np.where((grid_y <= Mlat) & (grid_y >= mlat))[0][[0, -1]]

	# Create a selection
	sel = [slice(Xspan[0], Xspan[1] + 1), slice(Yspan[0], Yspan[1] + 1)]
	# sel = [slice(Xspan[0] - 1, Xspan[1]), slice(Yspan[0] - 1, Yspan[1])]
	# sel = [slice(Yspan[0] -1, Yspan[1] + 1), slice(Xspan[0]-1, Xspan[1] + 1)]

	# grid_rain[t,sel[0],sel[1]] = storm_rain
	for i in range(20):
		storm_rain = storm.precipitation[t,:,:,i]
		grid_rain[t,sel[1],sel[0],i] = storm_rain
		storm_rain2 = storm2.precipitation[t,:,:,i]
		grid_rain2[t,sel[1],sel[0],i] = storm_rain2
	grid_rain_truth[t,sel[1],sel[0]] = storm_rain_truth
	

In [ ]:
total_storm_rain = np.sum(grid_rain[:,:,:,0],axis=0)
total_storm_rain2 = np.sum(grid_rain2[:,:,:,0],axis=0)
total_storm_rain_truth = np.sum(grid_rain_truth,axis=0)
rain_bool = total_storm_rain > 50
rain_bool2 = total_storm_rain2 > 50
print(np.sum(rain_bool))
rain_grid_x2 = grid_x[rain_bool2]

rain_bool_truth = total_storm_rain_truth > 50
print(np.sum(rain_bool_truth))
rain_grid_x_truth = grid_x[rain_bool_truth]

In [ ]:
hit = (rain_bool == True) & (rain_bool_truth == True)
print(np.sum(hit))
miss = (rain_bool == False) & (rain_bool_truth == True)
hit_rate = np.sum(hit) / (np.sum(hit) + np.sum(miss))
print('Hit rate: ',hit_rate*100)

false_alarm = (rain_bool == True) & (rain_bool_truth == False)
correct_rejection = (rain_bool == False) & (rain_bool_truth == False)
false_alarm_rate = np.sum(false_alarm)/(np.sum(false_alarm) + np.sum(correct_rejection))
print('false alarm rate',false_alarm_rate*100)

hit2 = (rain_bool2 == True) & (rain_bool_truth == True)
print(np.sum(hit))
miss2 = (rain_bool2 == False) & (rain_bool_truth == True)
hit_rate2 = np.sum(hit2) / (np.sum(hit2) + np.sum(miss2))
print('Hit rate2: ',hit_rate2*100)

false_alarm2 = (rain_bool2 == True) & (rain_bool_truth == False)
correct_rejection2 = (rain_bool2 == False) & (rain_bool_truth == False)
false_alarm_rate2 = np.sum(false_alarm2)/(np.sum(false_alarm2) + np.sum(correct_rejection2))
print('false alarm rate2',false_alarm_rate2*100)

# equitable threat score is:
# (hit - random chance) / (the observed rain area exceeding the criterion + model precticted area - hit - random chance)
# where R = the observed rain area exceeding the criterion * (model predicted area / entire verification domain)
# http://cimss.ssec.wisc.edu/goes/comet/threat.html
# ETS = (hits - E) / (hits + misses + false alarms - E)
# E = (number of forecast points * number of observed points) / total number of points possible

n_forecast_points = np.sum(rain_bool == True)
n_observed_points = np.sum(rain_bool_truth == True)
total_points_possible = np.sum((rain_bool_truth == True) | (rain_bool_truth == False))
E = (n_forecast_points*n_observed_points) / total_points_possible
ETS = (np.sum(hit) - E) / (np.sum(hit) + np.sum(miss) + np.sum(false_alarm) - E)
print('ETS: ', ETS)


n_forecast_points2 = np.sum(rain_bool2 == True)
n_observed_points2 = np.sum(rain_bool_truth == True)
total_points_possible2 = np.sum((rain_bool_truth == True) | (rain_bool_truth == False))
E = (n_forecast_points2*n_observed_points2) / total_points_possible2
ETS = (np.sum(hit2) - E) / (np.sum(hit2) + np.sum(miss2) + np.sum(false_alarm2) - E)
print('ETS2: ', ETS)
# The number of forecasts of the event correct by chance, a r , is determined by assuming that the forecasts are totally independent of the observations, and forecast will match the observation only by chance. This is one form of an unskilled forecast, which can be generated by just guessing what will happen. The ETS has a range of -1/3 to 1, but the minimum value depends on the verification sample climatology. For rare events, the minimum ETS value is near 0, while the absolute minimum is obtained if the event has a climatological frequency of 0.5, and there are no hits. If the score goes below 0 then the chance forecast is preferred to the actual forecast, and the forecast is said to be unskilled.

In [ ]:
def exposure_calc(population, rain,threshold):
	""" number of people exposed to over 200mm in New York"""
	# population[rain['precipitation'] < 200] = 0
	# print('max rain',np.max(rain['precipitation'].values))
	print('max rain',np.max(rain))
	# population = population.where(rain['precipitation'].values > threshold,0)
	population = population.where(rain > threshold,0)
	# exposure = np.nansum(population['Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 2.5 arc-minutes'].values[2])
	pop = population.population.values
	pop[pop == -inf] = 0
	exposure = np.nansum(pop)
	return exposure

In [ ]:
accumulated_rain = np.sum(grid_rain,axis=0)
accumulated_rain2 = np.sum(grid_rain2,axis=0)

In [ ]:
population_file = '/user/home/al18709/work/population/ppp_2020_10km_Aggregated_final.nc'
population_count = xr.load_dataset(population_file)
print(population_count)
data = population_count.population.values * 144
pop = population_count * 144


pop_lats = population_count.lat
pop_lons = population_count.lon
grid_x_pop, grid_y_pop = np.meshgrid(pop_lons,pop_lats)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
levels = range(1,1500000,20000)
# levels = [1,1000,5000,10000,100000,120000,160000,180000,200000,400000,600000,800000,1000000,2000000,3000000,4000000,5000000]
ax.contourf(lon,-lat + 12,data,levels=levels, transform=ccrs.PlateCarree(),cmap='Oranges')
ax.add_feature(cfeature.COASTLINE,linewidth=0.5)
ax.set_extent([-5, 5, 50, 60], crs=ccrs.PlateCarree())	
plt.show()

In [ ]:
# plt.imshow(data)
print(np.min(data))
data[data == -inf] = 0

In [ ]:
levels = [10, 15, 20, 25, 30, 40, 50,75, 100,140,180,200,250,300]
sns.set_style("white")
sns.set_context("notebook")
vmin=0
vmax=300


fig, ax = plt.subplots(figsize=(20,10),subplot_kw={'projection': ccrs.PlateCarree()})

ax.contourf(lon,-lat + 12,data,levels=range(1,200000,20000), transform=ccrs.PlateCarree(),cmap='Oranges')
ax.contourf(grid_x,grid_y,np.sum(grid_rain_truth,axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)
# ax.contourf(grid_x,grid_y,np.sum(grid_rain,axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)


ax.add_feature(cfeature.COASTLINE,linewidth=0.5)
ax.add_feature(cfeature.LAND, zorder=100,color='black',alpha=0.1)


# plt.contourf(storm.precipitation[0,:,:,0],storm.storm_lats[0,:,:],storm.storm_lons[0,:,:])


# ax.outline_patch.set_linewidth(0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
			linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 14}
gl.ylabel_style = {'size': 14}
# ax.set_xticklabels(labelsize=20)
# ax.set_yticklabels(labelsize=20)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

precip_cmap,precip_norm = make_cmap(high_vals=True)
# cbar = plt.colorbar(c,fraction=1.5, pad=-0.7,cmap=precip_cmap,ticks=levels,boundaries=levels, format='%1i',cax=ax)
# cbar.ax.tick_params(labelsize=8,width=0.5)
# cbar.ax.tick_params(labelsize=20)

ax.set_extent([grid_x[0,0], grid_x[-1,-1], grid_y[0,0], grid_y[-1,-1]], crs=ccrs.PlateCarree())
# ax.set_title('title',fontsize=26,pad=15)

plt.savefig('figure_7a_irma_rain.png',bbox_inches='tight')

In [ ]:
print(grid_x)
print(grid_y)

In [ ]:
# exposure
# storm_pop = pop.sel(lat=slice(-lats[-1]+1+12,-lats[0]+12), lon=slice(lons[0],lons[-1]+1))
# print(storm_pop)
# storm_pop_data = storm_pop.population.values
# print(storm_pop_data.shape)
# print(np.sum(grid_rain,axis=0).shape)
# print(grid_x.shape)
print(pop.population.shape)
accumulated_rain = np.sum(grid_rain[:,:,:,0],axis=0)
accumulated_rain_truth = np.sum(grid_rain_truth,axis=0)


# superimpose rain onto bigger grid
global_rain = np.zeros((pop.population.shape[0],pop.population.shape[1]))
global_rain_truth = np.zeros((pop.population.shape[0],pop.population.shape[1]))

storm_lons = grid_x
storm_lats = grid_y
storm_rain = accumulated_rain
storm_rain_truth = accumulated_rain_truth

Mlon = storm_lons[-1,-1]
mlon = storm_lons[0,0]
Mlat = storm_lats[-1,-1]
mlat = storm_lats[0,0]
grid_X,grid_Y = np.meshgrid(lon,lat)
Xspan = np.where((grid_X <= Mlon) & (grid_X >= mlon))[1][[0, -1]]
Yspan = np.where((grid_Y <= Mlat) & (grid_Y >= mlat))[0][[0, -1]]

# Create a selection
sel = [slice(Xspan[0], Xspan[1] + 1), slice(Yspan[0], Yspan[1] + 1)]

global_rain[sel[1],sel[0]] = storm_rain
global_rain_truth[sel[1],sel[0]] = storm_rain_truth



pred_exposure = exposure_calc(pop,global_rain,150)
truth_exposure = exposure_calc(pop,global_rain_truth,150)


In [ ]:
print(pred_exposure)
print(truth_exposure)

In [ ]:
plt.imshow(global_rain)

In [ ]:
def ets(total_storm_rain,total_storm_rain_truth,pop,threshold=50):

	rain_bool = total_storm_rain > threshold
	# rain_grid_x = grid_X[rain_bool]

	rain_bool_truth = total_storm_rain_truth > threshold
	# rain_grid_x_truth = grid_X[rain_bool_truth]

	hit = (rain_bool == True) & (rain_bool_truth == True)
	miss = (rain_bool == False) & (rain_bool_truth == True)
	hit_rate = np.sum(hit) / (np.sum(hit) + np.sum(miss))
	print('Hit rate: ',hit_rate*100)
	pop_hit = np.sum(pop[hit])
	pop_miss = np.sum(pop[miss])
	pop_hit_rate = pop_hit / (pop_hit + pop_miss)


	false_alarm = (rain_bool == True) & (rain_bool_truth == False)
	correct_rejection = (rain_bool == False) & (rain_bool_truth == False)
	false_alarm_rate = np.sum(false_alarm)/(np.sum(false_alarm) + np.sum(correct_rejection))
	print('false alarm rate',false_alarm_rate*100)
	pop_false_alarm = np.sum(pop[false_alarm])
	pop_correct_rejection = np.sum(pop[correct_rejection])
	pop_false_alarm_rate = pop_false_alarm / (pop_false_alarm + pop_correct_rejection)

	# equitable threat score is:
	# (hit - random chance) / (the observed rain area exceeding the criterion + model precticted area - hit - random chance)
	# where R = the observed rain area exceeding the criterion * (model predicted area / entire verification domain)
	# http://cimss.ssec.wisc.edu/goes/comet/threat.html
	# ETS = (hits - E) / (hits + misses + false alarms - E)
	# E = (number of forecast points * number of observed points) / total number of points possible

	n_forecast_points = np.sum(rain_bool == True)
	n_observed_points = np.sum(rain_bool_truth == True)
	total_points_possible = np.sum((rain_bool_truth == True) | (rain_bool_truth == False))
	E = (n_forecast_points*n_observed_points) / total_points_possible
	ETS = (np.sum(hit) - E) / (np.sum(hit) + np.sum(miss) + np.sum(false_alarm) - E)
	print('ETS: ', ETS)

	n_fp_pop = np.sum(pop[rain_bool])
	n_obs_pop = np.sum(pop[rain_bool_truth])
	bool = (rain_bool_truth == True) | (rain_bool_truth == False)
	total_pp_pop = np.sum(pop[bool])
	E_pop = (n_fp_pop*n_obs_pop) / total_pp_pop
	ETS_pop = (np.sum(pop_hit) - E_pop) / (np.sum(pop_hit) + np.sum(pop_miss) + np.sum(pop_false_alarm) - E_pop)
	print(ETS_pop)

	return hit_rate,false_alarm_rate,ETS,pop_hit,pop_miss,pop_hit_rate,pop_false_alarm,pop_correct_rejection,pop_false_alarm_rate
	# The number of forecasts of the event correct by chance, a r , is determined by assuming that the forecasts are totally independent of the observations, and forecast will match the observation only by chance. This is one form of an unskilled forecast, which can be generated by just guessing what will happen. The ETS has a range of -1/3 to 1, but the minimum value depends on the verification sample climatology. For rare events, the minimum ETS value is near 0, while the absolute minimum is obtained if the event has a climatological frequency of 0.5, and there are no hits. If the score goes below 0 then the chance forecast is preferred to the actual forecast, and the forecast is said to be unskilled.


In [ ]:
print(pop)

global_pop = pop.population.values

global_pop[global_pop == -inf] = 0

print(global_pop.shape)

In [ ]:
hit_rate,false_alarm_rate,ETS,pop_hit,pop_miss,pop_hit_rate,pop_false_alarm,pop_correct_rejection,pop_false_alarm_rate = ets(global_rain,global_rain_truth,global_pop,threshold=150)

In [ ]:
print(pop_hit_rate * 100)
print(pop_false_alarm_rate * 100)
print(grid_rain.shape)
print(grid_rain_truth.shape)


In [ ]:
n_time_steps,n,m,_ = grid_rain.shape
combo_rain = np.zeros((n_time_steps,n,m))
print(storm_scores.shape)
for t in range(n_time_steps):
	t_scores = storm_scores[t,:]
	best_score = np.argmax(t_scores)
	combo_rain[t,:,:] = grid_rain[t,:,:,best_score]

ensemble_combo = np.sum(combo_rain,axis=0)
print(t_scores)
print(ensemble_combo.shape)

In [ ]:
# levels = [10, 15, 20, 25, 30, 40, 50,75, 100,140,180,200,250,300]
# sns.set_style("white")
# sns.set_context("notebook")
# vmin=-2
# vmax=2

# TODO: add in combination of ensemble members. Not the mean, but based on the critic score of each image - e.g. the ones which are the best quality are the ones we select

levels = [10, 15, 20, 25, 30, 40, 50,75, 100,140,180,200,250,300]
sns.set_style("white")
sns.set_context("notebook")
vmin=0
vmax=300


fig, axes = plt.subplots(2,3,figsize=(20,16),subplot_kw={'projection': ccrs.PlateCarree()})
rain1 = np.sum(grid_rain[:,:,:,0],axis=0) > 50
rain2 = np.sum(grid_rain_truth,axis=0) > 50
hit = (rain1 == True) & (rain2 == True)
miss = (rain1 == False) & (rain2 == True)
false_alarm = (rain1 == True) & (rain2 == False)
correct_rejection = (rain1 == False) & (rain2 == False)
rain_data = hit + 2* miss + 3* false_alarm + 4*correct_rejection


# Create a colormap with distinct colors
colors = ['white', '#A3C9A9', '#C9A9A3', '#f2cc8f','white']
cmap = mcolors.ListedColormap(colors)

# Create color boundaries
bounds = np.arange(1, 5, 1)

# Create a BoundaryNorm instance
norm = mcolors.BoundaryNorm(bounds, cmap.N, extend='both')
# norm = mcolors.BoundaryNorm(bounds, cmap.N)

# first panel
axes[0,0].contourf(lon,-lat + 12,data,levels=range(1,200000,20000), transform=ccrs.PlateCarree(),cmap='Oranges')
m = axes[0,0].contourf(grid_x,grid_y,np.sum(grid_rain_truth,axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)

# second panel
axes[1,0].contourf(lon,-lat + 12,data,levels=range(1,200000,20000), transform=ccrs.PlateCarree(),cmap='Oranges')

agreement_range = 20
ensemble_max = np.max(np.sum(grid_rain[:,:,:,:],axis=0),axis=-1)
agree_mask = np.all([np.abs(np.sum(grid_rain[:,:,:,0],axis=0) - ensemble_max) <= agreement_range,
                    np.abs(np.sum(grid_rain[:,:,:,1],axis=0) - ensemble_max) <= agreement_range,
                    np.abs(np.sum(grid_rain[:,:,:,2],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain[:,:,:,3],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain[:,:,:,4],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain[:,:,:,5],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain[:,:,:,6],axis=0) - ensemble_max) <= agreement_range],
					axis=0)
print(agree_mask)
agree_mask = np.all([agree_mask,ensemble_max > 10],axis=0)
axes[0,1].contourf(grid_x,grid_y,ensemble_combo,vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)
axes[0,2].contourf(grid_x,grid_y,np.sum(grid_rain[:,:,:,0],axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)

axes[1,0].contourf(grid_x,grid_y,ensemble_max,vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)
axes[1,1].contourf(grid_x,grid_y,np.sum(grid_rain[:,:,:,0],axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)

stand_dev = np.std(np.sum(grid_rain[:,:,:,:],axis=0),axis=-1)

axes[1,1].contourf(grid_x,grid_y,stand_dev,vmin=0,vmax=40,levels=[0,5,10,15,20,30,40],cmap = 'Oranges', transform=ccrs.PlateCarree(),alpha=0.8)
# Create a hatching pattern
hatching_pattern = ['/', '\\', '//', '\\\\', '-', '|', '+', 'x', 'o', 'O', '.', '*'][-1]
# Add hatching to the areas of agreement
# axes[1].contourf(grid_x, grid_y, 1* agree_mask,1,hatches=['','////'],colors='none',transform=ccrs.PlateCarree())



h = axes[1,2].pcolormesh(grid_x,grid_y,rain_data,cmap=cmap, transform=ccrs.PlateCarree(), shading='auto', norm=norm)

for i in range(3):
	for j in range(2):
		axes[j,i].add_feature(cfeature.COASTLINE,linewidth=0.5)
		axes[j,i].add_feature(cfeature.LAND, zorder=100,color='black',alpha=0.1)


		# ax.outline_patch.set_linewidth(0.5)
		gl = axes[j,i].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
					linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
		gl.xlabels_top = False
		gl.ylabels_right = False
		gl.xlabel_style = {'size': 12}
		gl.ylabel_style = {'size': 12}
		# ax.set_xticklabels(labelsize=20)
		# ax.set_yticklabels(labelsize=20)
		axes[j,i].tick_params(axis='x', labelsize=14)
		axes[j,i].tick_params(axis='y', labelsize=14)

	# precip_cmap,precip_norm = make_cmap(high_vals=True)
	# cbar = plt.colorbar(h, ticks=np.arange(0.5, 4.5, 1), boundaries=bounds, label='Rain Category')
		axes[j,i].set_extent([grid_x[0,0], grid_x[-1,-1], grid_y[0,0], grid_y[-1,-1]], crs=ccrs.PlateCarree())
	# ax.set_title('Hit and Miss',fontsize=18,pad=15)

axes[0,0].text(-0.1, 1.05, 'a.', transform=axes[0,0].transAxes, size=18, weight='bold')
axes[0,1].text(-0.1, 1.05, 'b.', transform=axes[0,1].transAxes, size=18, weight='bold')
axes[0,2].text(-0.1, 1.05, 'c.', transform=axes[0,2].transAxes, size=18, weight='bold')
axes[1,0].text(-0.1, 1.05, 'd.', transform=axes[1,0].transAxes, size=18, weight='bold')
axes[1,1].text(-0.1, 1.05, 'e.', transform=axes[1,1].transAxes, size=18, weight='bold')
axes[1,2].text(-0.1, 1.05, 'f.', transform=axes[1,2].transAxes, size=18, weight='bold')
legend_labels = ['LEGEND', 'Hit', 'Miss', 'False alarm','Correct rejection']
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=color, markersize=10) for color, label in zip(colors, legend_labels)]
axes[1,2].legend(handles=legend_handles, loc='upper right')

axes[0,0].set_title('Truth',fontsize=18,pad=10)
axes[0,1].set_title('Predicted (critic combined ensemble)',fontsize=18,pad=10)
axes[0,2].set_title('Predicted (first ensemble member)',fontsize=18,pad=10)
axes[1,0].set_title('Predicted (ensemble max)',fontsize=18,pad=10)
axes[1,1].set_title('Standard deviation',fontsize=18,pad=10)
axes[1,2].set_title('Hit and Miss',fontsize=18,pad=10)

# cbar = plt.colorbar(m,fraction=1.5, pad=-0.7,cmap=precip_cmap,ticks=levels,boundaries=levels, format='%1i',cax=axes[0])
# cbar.ax.tick_params(labelsize=8,width=0.5)
# cbar.ax.tick_params(labelsize=20)

# cbar = plt.colorbar(m,fraction=1.5, pad=-0.7,cmap=precip_cmap,ticks=levels,boundaries=levels, format='%1i',cax=axes[1])
# cbar.ax.tick_params(labelsize=8,width=0.5)
# cbar.ax.tick_params(labelsize=20)


plt.savefig('figure_7b_hitmiss.png',bbox_inches='tight')

# TODO: compare hit miss and ets of combined score vs max and first,second,third etc

# TODO: rainfall is gamma distribution so std not that meaningful, so plot most probable rainfall (not mean) up to 95th percentile


In [ ]:
# TODO 
# the question is, what level of extreme rainfall can we be confident in calculating the exposure to?
# how often are we wrong when it's 50mm,100mm,150mm
# how often are we right?
# do the ensemble combining method with critic score so best of 20 images are selected


In [ ]:
# levels = [10, 15, 20, 25, 30, 40, 50,75, 100,140,180,200,250,300]
# sns.set_style("white")
# sns.set_context("notebook")
# vmin=-2
# vmax=2

# TODO: add in combination of ensemble members. Not the mean, but based on the critic score of each image - e.g. the ones which are the best quality are the ones we select

levels = [10, 15, 20, 25, 30, 40, 50,75, 100,140,180,200,250,300]
sns.set_style("white")
sns.set_context("notebook")
vmin=0
vmax=300


fig, axes = plt.subplots(2,3,figsize=(20,16),subplot_kw={'projection': ccrs.PlateCarree()})
rain1 = np.sum(grid_rain2[:,:,:,0],axis=0) > 50
rain2 = np.sum(grid_rain_truth,axis=0) > 50
hit = (rain1 == True) & (rain2 == True)
miss = (rain1 == False) & (rain2 == True)
false_alarm = (rain1 == True) & (rain2 == False)
correct_rejection = (rain1 == False) & (rain2 == False)
rain_data = hit + 2* miss + 3* false_alarm + 4*correct_rejection


# Create a colormap with distinct colors
colors = ['white', '#A3C9A9', '#C9A9A3', '#f2cc8f','white']
cmap = mcolors.ListedColormap(colors)

# Create color boundaries
bounds = np.arange(1, 5, 1)

# Create a BoundaryNorm instance
norm = mcolors.BoundaryNorm(bounds, cmap.N, extend='both')
# norm = mcolors.BoundaryNorm(bounds, cmap.N)

# first panel
axes[0,0].contourf(lon,-lat + 12,data,levels=range(1,200000,20000), transform=ccrs.PlateCarree(),cmap='Oranges')
m = axes[0,0].contourf(grid_x,grid_y,np.sum(grid_rain_truth,axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)

# second panel
axes[1,0].contourf(lon,-lat + 12,data,levels=range(1,200000,20000), transform=ccrs.PlateCarree(),cmap='Oranges')

agreement_range = 20
ensemble_max = np.max(np.sum(grid_rain2[:,:,:,:],axis=0),axis=-1)
agree_mask = np.all([np.abs(np.sum(grid_rain2[:,:,:,0],axis=0) - ensemble_max) <= agreement_range,
                    np.abs(np.sum(grid_rain2[:,:,:,1],axis=0) - ensemble_max) <= agreement_range,
                    np.abs(np.sum(grid_rain2[:,:,:,2],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain2[:,:,:,3],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain2[:,:,:,4],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain2[:,:,:,5],axis=0) - ensemble_max) <= agreement_range,
					np.abs(np.sum(grid_rain2[:,:,:,6],axis=0) - ensemble_max) <= agreement_range],
					axis=0)
print(agree_mask)
agree_mask = np.all([agree_mask,ensemble_max > 10],axis=0)
axes[0,1].contourf(grid_x,grid_y,ensemble_combo,vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)
axes[0,2].contourf(grid_x,grid_y,np.sum(grid_rain2[:,:,:,0],axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)

axes[1,0].contourf(grid_x,grid_y,ensemble_max,vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)
axes[1,1].contourf(grid_x,grid_y,np.sum(grid_rain2[:,:,:,0],axis=0),vmin=vmin,vmax=vmax,levels=levels,cmap = precip_cmap, transform=ccrs.PlateCarree(),alpha=0.8)

stand_dev = np.std(np.sum(grid_rain2[:,:,:,:],axis=0),axis=-1)

axes[1,1].contourf(grid_x,grid_y,stand_dev,vmin=0,vmax=40,levels=[0,5,10,15,20,30,40],cmap = 'Oranges', transform=ccrs.PlateCarree(),alpha=0.8)
# Create a hatching pattern
hatching_pattern = ['/', '\\', '//', '\\\\', '-', '|', '+', 'x', 'o', 'O', '.', '*'][-1]
# Add hatching to the areas of agreement
# axes[1].contourf(grid_x, grid_y, 1* agree_mask,1,hatches=['','////'],colors='none',transform=ccrs.PlateCarree())



h = axes[1,2].pcolormesh(grid_x,grid_y,rain_data,cmap=cmap, transform=ccrs.PlateCarree(), shading='auto', norm=norm)

for i in range(3):
	for j in range(2):
		axes[j,i].add_feature(cfeature.COASTLINE,linewidth=0.5)
		axes[j,i].add_feature(cfeature.LAND, zorder=100,color='black',alpha=0.1)


		# ax.outline_patch.set_linewidth(0.5)
		gl = axes[j,i].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
					linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
		gl.xlabels_top = False
		gl.ylabels_right = False
		gl.xlabel_style = {'size': 12}
		gl.ylabel_style = {'size': 12}
		# ax.set_xticklabels(labelsize=20)
		# ax.set_yticklabels(labelsize=20)
		axes[j,i].tick_params(axis='x', labelsize=14)
		axes[j,i].tick_params(axis='y', labelsize=14)

	# precip_cmap,precip_norm = make_cmap(high_vals=True)
	# cbar = plt.colorbar(h, ticks=np.arange(0.5, 4.5, 1), boundaries=bounds, label='Rain Category')
		axes[j,i].set_extent([grid_x[0,0], grid_x[-1,-1], grid_y[0,0], grid_y[-1,-1]], crs=ccrs.PlateCarree())
	# ax.set_title('Hit and Miss',fontsize=18,pad=15)

axes[0,0].text(-0.1, 1.05, 'a.', transform=axes[0,0].transAxes, size=18, weight='bold')
axes[0,1].text(-0.1, 1.05, 'b.', transform=axes[0,1].transAxes, size=18, weight='bold')
axes[0,2].text(-0.1, 1.05, 'c.', transform=axes[0,2].transAxes, size=18, weight='bold')
axes[1,0].text(-0.1, 1.05, 'd.', transform=axes[1,0].transAxes, size=18, weight='bold')
axes[1,1].text(-0.1, 1.05, 'e.', transform=axes[1,1].transAxes, size=18, weight='bold')
axes[1,2].text(-0.1, 1.05, 'f.', transform=axes[1,2].transAxes, size=18, weight='bold')
legend_labels = ['LEGEND', 'Hit', 'Miss', 'False alarm','Correct rejection']
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=color, markersize=10) for color, label in zip(colors, legend_labels)]
axes[1,2].legend(handles=legend_handles, loc='upper right')

axes[0,0].set_title('Truth',fontsize=18,pad=10)
axes[0,1].set_title('Predicted (critic combined ensemble)',fontsize=18,pad=10)
axes[0,2].set_title('Predicted (first ensemble member)',fontsize=18,pad=10)
axes[1,0].set_title('Predicted (ensemble max)',fontsize=18,pad=10)
axes[1,1].set_title('Standard deviation',fontsize=18,pad=10)
axes[1,2].set_title('Hit and Miss (<50 mm)',fontsize=18,pad=10)

# cbar = plt.colorbar(m,fraction=1.5, pad=-0.7,cmap=precip_cmap,ticks=levels,boundaries=levels, format='%1i',cax=axes[0])
# cbar.ax.tick_params(labelsize=8,width=0.5)
# cbar.ax.tick_params(labelsize=20)

# cbar = plt.colorbar(m,fraction=1.5, pad=-0.7,cmap=precip_cmap,ticks=levels,boundaries=levels, format='%1i',cax=axes[1])
# cbar.ax.tick_params(labelsize=8,width=0.5)
# cbar.ax.tick_params(labelsize=20)


plt.savefig('figure_7c_hitmiss_2.png',bbox_inches='tight')

# TODO: compare hit miss and ets of combined score vs max and first,second,third etc

# TODO: rainfall is gamma distribution so std not that meaningful, so plot most probable rainfall (not mean) up to 95th percentile
